**Import** *and* setup some auxiliary functions

In [0]:
# Don't edit this cell
import os
import timeit
import time
import numpy as np
from collections import OrderedDict
from pprint import pformat
from tqdm import tqdm
from google.colab import drive

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data.sampler import *
from torchvision import transforms, datasets

torch.multiprocessing.set_sharing_strategy('file_system')
cudnn.benchmark = True


In [0]:
# TODO: Main model definition + any utilities such as weight initialization or custom layers, ADD DROPOUT, BATCHNORM, SKIP CONNECTION,
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 52, kernel_size=5)       
    self.conv2 = nn.Conv2d(52, 146, kernel_size=5)      
    self.fc1 = nn.Linear(146 * 5 * 5, 840) 
    self.fc2 = nn.Linear(840, 520)
    self.fc3 = nn.Linear(520, 10)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.bn1 = nn.BatchNorm2d(146)
    self.bn2 = nn.BatchNorm1d(840)
    self.bn3 = nn.BatchNorm1d(520)
    self.dp = nn.Dropout(p=0.2)
    # Skip Connections?: to match the residual size and the input size, we need extra bn and conv2d
    self.sc = nn.Conv2d(3, 52, kernel_size=5)
    self.bnsc = nn.BatchNorm2d(52)
    
    # Xavier initialization
    torch.nn.init.xavier_normal_(self.conv1.weight)
    torch.nn.init.xavier_normal_(self.conv2.weight)
    torch.nn.init.xavier_normal_(self.fc1.weight)
    torch.nn.init.xavier_normal_(self.fc2.weight)
    torch.nn.init.xavier_normal_(self.fc3.weight)
    
  def forward(self, x):
    res = x
    x = F.relu(self.conv1(x))
    x += self.bnsc(self.sc(res))
    x = self.pool(x)
    x = self.pool(F.relu(self.bn1(self.conv2(x))))
    
    x = x.view(-1, 146 * 5 * 5)
    x = F.relu(self.bn2(self.fc1(x)))
    x = self.dp(F.relu(self.bn3(self.fc2(x))))
    x = F.log_softmax(self.fc3(x))
    return x

In [0]:
# TODO: Cifar-10 dataloading
def load_data(config):
    """
    Load cifar-10 dataset using torchvision, take the last 10k of the training data to be validation data
    """
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    # Import batch_size from config
    batch_size = config['batch_size']
    
    # Setup datasets
    CIFAR10_training = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    CIFAR10_test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    
    # create a training and a validation set by spliting training dataset to first 45000 and last 5000 indices
    num_train = len(CIFAR10_training)
    indices = list(range(num_train))
    train_idx, valid_idx = indices[:45000], indices[45000:]
    
    # Setup dataloaders, batch_size for the test loader should be 1
    train_sampler = SubsetRandomSampler(train_idx) 
    validation_sampler = SubsetRandomSampler(valid_idx)
    
    train_dataloader = torch.utils.data.DataLoader(CIFAR10_training,batch_size=batch_size, sampler=train_sampler, num_workers=2)
    valid_dataloader = torch.utils.data.DataLoader(CIFAR10_training,batch_size=batch_size, sampler=validation_sampler, num_workers=2)
    test_dataloader = torch.utils.data.DataLoader(CIFAR10_test_set,batch_size=1, shuffle=False, num_workers=2)
    
    
    return train_dataloader, valid_dataloader, test_dataloader

In [0]:
# TODO : Main trainig + validation, returns the final model, save your best checkpoint based on the best validation accuracy
def train(train_dataloader, valid_dataloader, device, config):
  # Import parameters from config
  lr = config['lr']
  momentum = config['momentum']
  num_epochs = config['num_epochs']
  regular_constant = config['regular_constant']
  log_interval = config['log_interval']
  
  # Setup model, loss function, optimizer, best accuracy and current accuracy
  model = Net().to(device)
  best_acc, curr_acc = 0, 0
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=regular_constant)
  
  # Per Iterations training + validation
  for epoch in range(1, num_epochs + 1):
    # Training
    for batch_idx, (data, target) in enumerate(train_dataloader):
      data = data.to(device)
      target = target.to(device)
      # Forward -> Backprop -> Optimize
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      
      if batch_idx % log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_dataloader.sampler),
          100. * batch_idx / len(train_dataloader), loss.item()))
    
    # Validation with 5000 dataset
    validation_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in valid_dataloader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        validation_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    validation_loss /= len(valid_dataloader.sampler)
    print('\nValidation set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(valid_dataloader.sampler),
        100. * correct / len(valid_dataloader.sampler)))
    
    # Save the checkpoint with the best accuracy
    curr_acc = 100. * correct / len(valid_dataloader.sampler)
    if curr_acc >= best_acc:
      save_checkpoint(model)
      best_acc = curr_acc
      print('(Current accuracy >= Best accuracy) ===> Model has been updated to the checkpoint\n')
      
  
  return model

In [0]:
# A simple function for save model to the checkpoint
def save_checkpoint(model):
  #drive.mount('/content/gdrive/', force_remount=True)
  torch.save(model.state_dict(), '/content/gdrive/My Drive/checkpoint/ckpt.pth')
  
def save_model_colab_for_submission(model):  # if you are running on colab
  #drive.mount('/content/gdrive/', force_remount=True)
  torch.save(model.to(torch.device("cpu")), '/content/gdrive/My Drive/model.pt') # you will find the model in your home drive
  
def save_model_local_for_submission(model):  # if you are running on your local machine
  torch.save(model.to(torch.device("cpu")), 'model.pt')

In [0]:
#TODO: Implement testing
def test(model, test_dataloader, device):
  correct = 0
  total = 0
  
  test_loss = 0
  criterion = nn.CrossEntropyLoss()
  
  with torch.no_grad():
    for data, target in test_dataloader:
      data = data.to(device)
      target = target.to(device)
      output = model(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
      
  test_loss /= len(test_dataloader.dataset)
  total = len(test_dataloader.dataset)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_dataloader.dataset),
    100. * correct / len(test_dataloader.dataset)))

  return 100.*correct/total, correct, total

In [0]:
def run():
  # Mount the google drive
  drive.mount('/content/gdrive/', force_remount=True)
  
  # set parameters cifar10
  config = {
        'lr': 1e-2,
        'num_epochs': 20,
        'batch_size': 128,
        'num_classes': 10,
        'momentum': 0.9,
        'regular_constant': 1e-4,
        'log_interval': 100,
       }
    
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  
  train_dataloader, valid_dataloader, test_dataloader = load_data(config)
  
  model = train(train_dataloader, valid_dataloader, device, config)
  
  # Testing and saving for submission
  device = torch.device("cpu")
  
  # Modified for google drive use
  #assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
  #checkpoint = torch.load('./checkpoint/ckpt.pth')
  
  checkpoint = torch.load('/content/gdrive/My Drive/checkpoint/ckpt.pth')
  model.load_state_dict(checkpoint)
  model.eval()
  
  start_time = timeit.default_timer()
  test_acc, test_correct, test_total = test(model.to(device), test_dataloader, device)
  end_time = timeit.default_timer()
  test_time = (end_time - start_time)
  
  save_model_colab_for_submission(model)

  return test_acc, test_correct, test_time

  

Main loop. Run time and total score will be shown below.

In [16]:
# Don't edit this cell
def compute_score(acc, min_thres=65, max_thres=80):
  # Your Score thresholds
  if acc <= min_thres:
      base_score = 0.0
  elif acc >= max_thres:
      base_score = 100.0
  else:
      base_score = float(acc - min_thres) / (max_thres - min_thres) * 100
  return base_score

def main():
    
    accuracy, correct, run_time = run()
    
    score = compute_score(accuracy)
    
    result = OrderedDict(correct=correct,
                         accuracy=accuracy,
                         run_time=run_time,
                         score=score)
    
    with open('result.txt', 'w') as f:
        f.writelines(pformat(result, indent=4))
    print("\nResult:\n", pformat(result, indent=4))


main()

Mounted at /content/gdrive/
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/45000 (0%)]	Loss: 2.919630
Train Epoch: 1 [12800/45000 (28%)]	Loss: 1.654056
Train Epoch: 1 [25600/45000 (57%)]	Loss: 1.393146
Train Epoch: 1 [38400/45000 (85%)]	Loss: 1.371235

Validation set: Avg. loss: 0.0106, Accuracy: 2627/5000 (53%)

(Current accuracy >= Best accuracy) ===> Model has been updated to the checkpoint

Train Epoch: 2 [0/45000 (0%)]	Loss: 1.253615
Train Epoch: 2 [12800/45000 (28%)]	Loss: 1.053297
Train Epoch: 2 [25600/45000 (57%)]	Loss: 1.320375
Train Epoch: 2 [38400/45000 (85%)]	Loss: 1.132117

Validation set: Avg. loss: 0.0094, Accuracy: 2913/5000 (58%)

(Current accuracy >= Best accuracy) ===> Model has been updated to the checkpoint

Train Epoch: 3 [0/45000 (0%)]	Loss: 1.204578
Train Epoch: 3 [12800/45000 (28%)]	Loss: 1.289159
Train Epoch: 3 [25600/45000 (57%)]	Loss: 1.255394
Train Epoch: 3 [38400/45000 (85%)]	Loss: 1.064853

Validation set: Avg. loss: 0.0082, Accuracy: 3171/5000 (63%)

(Current accuracy >= Best accuracy) ===> Model has been upda

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type MaxPool2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pa